## dF/F Traces from Caiman Analysis

This script enables you to plot the dF/F traces obtained from running caiman analyis with a .tif file and stores them in the NWB file created from Caiman analysis.

reference in making this script: https://allensdk.readthedocs.io/en/v2.2.0/_modules/allensdk/brain_observatory/nwb.html 

The first step is to load and read in the NWB file created from running the caiman analysis as well as the stored array of dF/F values.

In [ ]:
from pynwb import NWBHDF5IO
from pynwb.ophys import DfOverF

io = NWBHDF5IO("/Users/sayeholoumi/caiman_data/example_movies/m3_00002_estimates_cropped3v2.nwb","r+")

In [ ]:
nwb = io.read()

In [ ]:
%store -r dFOverF_cropped3v2

In [ ]:
dFOverF_cropped3v2.shape


You can now plot the dF/F trace.

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(dFOverF_cropped3v2[:,0:40])
plt.show()

The following code segment enables you to view the values of the numpy array.

In [ ]:
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)
print(dFOverF_cropped3v2[:,0:100])

In order to write the dF/F values into the NWB file, it is important to create a new data interface within the NWB file.

In [ ]:
dff_interface = DfOverF(name='dff')

In [ ]:
nwb.processing['ophys'].add_data_interface(dff_interface)

You can view the dataframe of the ROI IDs and their associated R, SNR, and CNN values by executing the code below.
The CNN and SNR values match with the ROIs displayed when running Caiman on the tif file. Please that the last two are spatial backgrounds (neuropil) of the tif file.

In [ ]:
df=nwb.processing['ophys'].get_data_interface('ImageSegmentation').get_plane_segmentation().to_dataframe()

In [ ]:
print(df)

The following line of code enables you to view the image mask of the neurons.

In [ ]:
mask=df["image_mask"][1]
print(type(mask))
print(mask.shape)
plt.imshow(mask.T)
plt.show()

The following code segment enables you to view the other contents of the processing module of the NWB file.

In [ ]:
print(nwb.processing['ophys'].get_data_interface('ImageSegmentation').get_plane_segmentation())

The next code segments will write the dF/F values to their corresponding ROIs in the table.

In [ ]:
rt_region = nwb.processing['ophys'].get_data_interface('ImageSegmentation').get_plane_segmentation().create_roi_table_region(
    description='segmented cells with cell_specimen_ids',
)
print(rt_region.to_dataframe())

In [ ]:
df_over_f = nwb.processing["ophys"].data_interfaces["Fluorescence"].roi_response_series["RoiResponseSeries"]

df_over_f_values = df_over_f.data[:]
timestamps = df_over_f.timestamps[:]

In [ ]:
dFF_series = dff_interface.create_roi_response_series(
    name='df_over_f',
    data=dFOverF_cropped3v2,
    unit='NA',
    rois=rt_region,
    timestamps=timestamps,
)

Now, you can write back the changes made to the nwbfile.

In [ ]:
io.write(nwb)
io.close()